<a href="https://colab.research.google.com/github/kiringki/DataProject/blob/main/MiniProject_02_SunnyAnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[미니프로젝트 2]  머신러닝을 활용한 Santander 고객 만족도 예측**

---

## 1. 과제 개요

본 프로젝트는 **스페인의 글로벌 금융기관 Santander**에서 제공한 고객 데이터를 기반으로,  
**고객의 만족 여부를 예측**하는 분류 문제입니다.

주어진 데이터에는 고객의 개인 정보, 계좌 활동, 신용 정보 등이 포함되어 있으며,  
이러한 특성을 활용하여 고객이 만족(1)하는지, 불만족(0)하는지를 예측하는 모델을 개발하는 것이 목표입니다.

---

## 2. 분석 목표

- 고객의 다양한 특성 데이터를 바탕으로 **만족 여부를 이진 분류**
- 분류 성능을 높이기 위한 **전처리 및 피처 엔지니어링 전략 수립**
- 예측 결과를 바탕으로 **고객 이탈 방지 및 만족도 향상**을 위한 인사이트 도출

---

## 3. 분석 절차 (일반적인 process)
- <font color="blue">본 프로젝트에서는 머신러닝의 프로세스를 명확히 인지하는 것이 중요하므로, 데이터 EDA 및 전처리 과정은 개인의 선택에 따라 자유롭게 진행하도록 함.<font color="black">

1. 데이터 탐색 및 결측치 확인
2. 피처 분포 분석 및 이상치 제거
3. 불필요한 피처 제거 및 파생 피처 생성
4. 학습/검증용 데이터 분할 (train/test split 또는 cross-validation)
5. 모델 학습 및 예측

---

## 4. 모델링 지침

- 반드시 다음 두 가지 모델을 사용하여 비교 분석합니다:
  -  `XGBoost (XGBClassifier)`
  -  `LightGBM (LGBMClassifier)`

- 성능 평가지표는 다음을 기준으로 합니다:
  - **Accuracy**
  - **F1-Score**
  - **ROC AUC Score**

- 모델의 **하이퍼파라미터 튜닝**을 통해 성능을 개선해보세요. (선택사항)

<br>

---

<br>


## **0. 라이브러리 불러오기**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/국방AI교육/대면교육/11. 미니프로젝트 2차

/content/drive/MyDrive/국방AI교육/대면교육/11. 미니프로젝트 2차


In [46]:
import numpy as np
import pandas as pd

# 데이터 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# 데이터 전처리(스케일링)
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# 데이터 분리
from sklearn.model_selection import train_test_split

# 모델링
import xgboost
import lightgbm

# 성능평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve


# setting
import warnings
warnings.filterwarnings('ignore')




## **1. 데이터 불러오기**

- 파일명 : santander.csv
- encoding : 'latin-1'   을 활용하세요  
- 데이터의 이름을 ```cust_df``` 로 불러오세요 (아래 전처리코드를 활용하기 위함)

In [47]:
## 데이터를 불러오고 확인


cust_df = pd.read_csv('santander.csv', encoding='latin-1')



## **2. 데이터 살펴보기**

- 위에서 불러온 데이터를 자유롭게 탐색해보세요

In [48]:
cust_df.shape

(76020, 371)

In [49]:
cust_df.head()


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [50]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [51]:
print(cust_df['TARGET'].value_counts())

unsatisfied_cnt = cust_df[cust_df['TARGET'] == 1].TARGET.count()
total_cnt = cust_df.TARGET.count()

print('unsatisfied 비율은 {0:.2f}'.format((unsatisfied_cnt / total_cnt)))

TARGET
0    73012
1     3008
Name: count, dtype: int64
unsatisfied 비율은 0.04


## **3. 데이터 전처리하기**

- 전처리를 위해 아래의 코드를 실행하세요(추가로 필요한 코드는 없습니다)
- 추가로 필요하다고 생각되는 전처리는 자유롭게 적용하세요(필수는 아닙니다)

In [52]:
# var3 피처 값 대체 및 ID 피처 드롭
cust_df['var3'].replace(-999999, 2, inplace=True)
cust_df.drop('ID', axis=1, inplace=True)

## **4. 데이터 분리하기**

- 독립변수와 종속변수를 나누세요
- 종속변수의 이름은 ```TARGET``` 입니다.
- 전체 데이터를 학습용/훈련용 데이터로 각각 나누세요 (옵션 자유)
- <font color="blue">학습용 데이터를 다시 학습용/검증용 데이터로 각각 나누세요.  <font color="black">(XGB, LightGBM 에서 검증용 옵션으로 사용하기 위함)






In [53]:
# 독립변수(X)와 종속변수(y)로 나누기 // 종속변수는 Target의 값, 그래서 iloc -1

X_features = cust_df.iloc[:, :-1]
y_labels = cust_df.iloc[:, -1]

In [54]:
## 전체 데이터를 학습용/훈련용 데이터로 각각 나누기

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)

#실제 테스트때는 _test 값을 사용

In [56]:
## 학습용 데이터를 다시 학습용/검증용 데이터로 각각 나누기

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

#실제 테스트때는 _val 값을 사용

## **5. 모델 학습과 예측 1 - XGBboost**

- ```XGBboost``` 모델을 학습용 데이터로 훈련시키세요
- ```XGBboost``` 옵션을 활용하여 검증용 데이터도 활용하세요
- 테스트용 데이터를 활용하여 acuracy, f1-score, roc-auc score 로 모델을 성능평가하세요.

In [98]:
## 모델을 객체화하고 학습

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score



xgb = XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=3, random_state=156)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_val)
pred_proba =xgb.predict_proba(X_val)[:, 1]


In [99]:
## 모델의 성능 평가 함수

def get_clf_eval(y_val, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_val, pred)
    accuracy = accuracy_score(y_val, pred)
    precision = precision_score(y_val, pred)
    recall = recall_score(y_val, pred)
    f1 = f1_score(y_val,pred)
    # ROC-AUC 추가
    roc_auc = roc_auc_score(y_val, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [100]:
#검증용 데이터로 평가

get_clf_eval(y_val , pred, pred_proba)

오차 행렬
[[9371    1]
 [ 357    2]]
정확도: 0.9632, 정밀도: 0.6667, 재현율: 0.0056,    F1: 0.0110, AUC:0.8138


In [101]:
#테스트용 데이터로 평가

pred_test = xgb.predict(X_test)
pred_test_proba =xgb.predict_proba(X_test)[:, 1]

get_clf_eval(y_test , pred_test, pred_test_proba)

오차 행렬
[[14565     5]
 [  633     1]]
정확도: 0.9580, 정밀도: 0.1667, 재현율: 0.0016,    F1: 0.0031, AUC:0.8397


## **6. 모델 학습과 예측 2 - LightGBM**

- ```LightGBM``` 모델을 학습용 데이터로 훈련시키세요
- ```LightGBM``` 옵션을 활용하여 검증용 데이터도 활용하세요
- 테스트용 데이터를 활용하여 acuracy, f1-score, roc-auc score 로 모델을 성능평가하세요.

In [102]:
## 모델을 객체화하고 학습

from lightgbm import LGBMClassifier

evals = [(X_val, y_val)]
lgbm = LGBMClassifier(n_estimators=500, verbose=-1)
lgbm.fit(X_train, y_train, eval_set=evals, eval_metric='logloss')
pred = lgbm.predict(X_val)
pred_proba = lgbm.predict_proba(X_val)[:, 1]

In [103]:
#검증용 데이터로 평가

get_clf_eval(y_val , pred, pred_proba)

오차 행렬
[[9363    9]
 [ 353    6]]
정확도: 0.9628, 정밀도: 0.4000, 재현율: 0.0167,    F1: 0.0321, AUC:0.7688


In [104]:
#테스트용 데이터로 평가

pred_test = lgbm.predict(X_test)
pred_test_proba =lgbm.predict_proba(X_test)[:, 1]

get_clf_eval(y_test , pred_test, pred_test_proba)

오차 행렬
[[14545    25]
 [  625     9]]
정확도: 0.9572, 정밀도: 0.2647, 재현율: 0.0142,    F1: 0.0269, AUC:0.8056


## **7. 모델의 성능향상(선택사항)**

- 모델의 성능향상을 위한 기법을 자유롭게 적용해보세요

### **<2개의 모델을 성능 비교하고 결과를 간단히 말해주세요>**


In [ ]:
## 여기에 성능 비교 결과를 써주세요

# 데이터의 값이 워낙 중구난방이라 F1값과 정밀도가 매우 낮지만, 그 와중에 AUC값을 비교해보았을 때,
# XGBboost 의 AUC와 F1값이 LGBM보다 약 3% ~ 5% 더 높아서 더 신뢰할 수 있는 결과라고 해석할 수 있다.
# 다만, 실행 시간은 XGB 17초, LGBM 6초로 현재는 데이터 값이 크지 않아 큰 문제가 되지 않았지만 실행 시간 측면에서 LGBM 이 낫다.
# 시간이 얼마 없을때 큰 데이터 혹은 같은 데이터 값을 연산해야한다면 LGBM이 선택하는 것이 시간을 소요 시간을 단축할 수 있다.